<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:250%;
           text-align:center;
           border-radius:10px 10px;"> 
    🌐 תהליך הסבת שכבת חלקות (דו מימדיות) לבנק"ל מודרני 🌐

In [ ]:
print('16. Parcels borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from configs import CNFG
import numpy as np
import pandas as pd
import geopandas as gpd
import dask_geopandas as daskgpd
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

<h1 style="text-align:right;font-size:125%">
חלקות פעילות

In [ ]:
print('  16.a Reading inputs')

In [ ]:
parcel_all_cols = ['Identifier', 'GUSH_NUM', 'GUSH_SUFFIX', 'PARCEL', 'STATUS', 'LEGAL_AREA', 'TALAR_NUMBER', 'TALAR_YEAR', 'geometry']

parcel_all = daskgpd.read_file(CNFG.Arcstorm + 'PARCEL_ALL.gdb', layer = 'PARCEL_ALL', columns = parcel_all_cols, npartitions = 20).compute()

del parcel_all_cols

print('         input 1/5 read')

<h1 style="text-align:right;font-size:125%">
טבלת מספרי זיהוי של גושים

In [ ]:
Blocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = ['Name', 'BlockUniqueID'])

print('         input 2/5 read')

<h1 style="text-align:right;font-size:125%">
נתוני גבולות תהליכי קדסטר

In [ ]:
cpb_columns = ['CPBUniqueID', 'GlobalID', 'ProcessType', 'ProcessName', 'Status', 'MapYear']

cpb = daskgpd.read_file(CNFG.ProductPath, layer = 'CadasterProcessBorders', columns = cpb_columns, npartitions = 3).compute()

del cpb_columns

print('         input 3/5 read')

<h1 style="text-align:right;font-size:125%">
טבלת סדר פעולות

In [ ]:
sequence_columns = ['ToParcelFinal', 'CPBUniqueID', 'ProcessType', 'ProcessName', 'BlockNumber', 'SubBlockNumber', 'ToBlockNumber', 'ToSubBlockNumber']

sequence = daskgpd.read_file(CNFG.ProductPath, layer = 'SequenceActions', columns = sequence_columns, npartitions = 25).compute()

del sequence_columns

print('         input 4/5 read')

<h1 style="text-align:right;font-size:125%">
נתוני מקרקעי ייעוד

In [ ]:
SubBlocksList = pd.read_csv(CNFG.Local + 'SubBlocksList.csv')
xml_df = pd.read_csv(CNFG.Local + 'KeshetRimonAssets.csv').sort_values(['BlockName', 'ParcelNumber'])

xml_df = xml_df.merge(SubBlocksList[['BlockNumber', 'SubBlockNumber', 'BlockName']], on = 'BlockName', how = 'left')

del SubBlocksList

print('         input 5/5 read')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

<h1 style="text-align:right;font-size:125%;">
:עיבוד נתוני חלקות פעילות

<h1 style="text-align:right;font-size:100%">
שינוי מרשת קואורדינטות עולמית לרשת ישראל חדשה  

In [ ]:
parcel_all = parcel_all.to_crs(2039)

<h1 style="text-align:right;font-size:100%">
איחוד עם טבלת גושים לקבלת מספר מזהה לגוש עבור כל חלקה

In [ ]:
parcel_all['BlockName'] = parcel_all['GUSH_NUM'].astype(str) + '/' + parcel_all['GUSH_SUFFIX'].astype(str)
parcel_all = parcel_all.merge(Blocks, left_on = 'BlockName', right_on = 'Name', how = 'left')
parcel_all.drop(columns = ['BlockName', 'Name'], inplace = True)
del Blocks

<h1 style="text-align:right;font-size:100%">
שינוי שמות של שדות ישנים לחדשים

In [ ]:
rename_dict = {'PARCEL'      : 'ParcelNumber',
               'GUSH_NUM'    : 'BlockNumber',
               'GUSH_SUFFIX' : 'SubBlockNumber',
               'Identifier'  : 'ParcelUniqueID',
               'LEGAL_AREA'  : 'StatedArea'}


parcel_all.rename(columns = rename_dict, inplace = True)

<h1 style="text-align:right;font-size:100%">
הגדרת שדה של סוג מקרקעין

In [ ]:
registered   = 1
unregistered = 2

registered_cond = parcel_all['STATUS'].isin([1, 3, 6, 16])
parcel_all['LandType'] = np.where(registered_cond, registered, unregistered)
parcel_all.drop(columns = 'STATUS', inplace = True)

<h1 style="text-align:right;font-size:100%">
הגדרת שדה של סוג חלקה

In [ ]:
temporary  = 1
final      = 2

parcel_all['ParcelType'] = parcel_all['LandType'].map({registered : final, unregistered : temporary})

del [registered, unregistered, temporary, final]

<h1 style="text-align:right;font-size:100%">
הגדרת שדה של יחידת שטח למטר מרובע

In [ ]:
parcel_all['StatedAreaUnit'] = 109404

<h1 style="text-align:right;font-size:100%">
מיזוג נתוני מקרקעי יעוד

In [ ]:
xml_df['BlockNumber'] = xml_df['BlockNumber'].fillna(xml_df['BlockName'])

xml_df['SubBlockNumber'] = xml_df['SubBlockNumber'].fillna(0)

xml_df = xml_df.drop(columns = 'BlockName')

xml_df[['BlockNumber', 'SubBlockNumber', 'ParcelNumber']] = xml_df[['BlockNumber', 'SubBlockNumber', 'ParcelNumber']].astype(int)

xml_df['ParcelName'] = xml_df['ParcelNumber'].astype(str) + '/' + xml_df['BlockNumber'].astype(str) + '/' +  xml_df['SubBlockNumber'].astype(str)

xml_df = xml_df[['ParcelName', 'LandDesignations']]



parcel_all['ParcelName'] = parcel_all['ParcelNumber'].astype(str) + '/' + parcel_all['BlockNumber'].astype(str) + '/' + parcel_all['SubBlockNumber'].astype(str)



parcel_all = parcel_all.merge(right = xml_df, on = 'ParcelName', how = 'left')
parcel_all.rename(columns = {'LandDesignations' : 'LandDesignationPlan'}, inplace = True)

del xml_df

<h1 style="text-align:right;font-size:100%">
סינון נתוני תהליכי קדסטר לתכניות שאושרו בלבד (חלקות רצף הן חלקות לאחר שאושרו/עודכנו בתכנית)

In [ ]:
ApprovedStatus = [5, 6, 13, 94, 98, 102, 103]

cpb = cpb[cpb['Status'].isin(ApprovedStatus)]
#cpb = cpb.drop(columns = 'Status')

del ApprovedStatus

<h1 style="text-align:right;font-size:100%">
עיבוד טבלת סדר פעולות 

In [ ]:
sequence['ToBlockNumber'] = sequence['ToBlockNumber'].fillna(sequence['BlockNumber'])
sequence['ToSubBlockNumber'] = sequence['ToSubBlockNumber'].fillna(sequence['SubBlockNumber'])

sequence = sequence.astype('Int64')

sequence = sequence.merge(cpb, on =['CPBUniqueID', 'ProcessType'], how = 'left', validate = 'many_to_one')

sequence = sequence[
                    (sequence['ToParcelFinal'] !=  0)
                                  &
                    (~sequence['ToParcelFinal'].isna())
                                  &
                    (sequence['ToBlockNumber'] !=  0)
                    ]

sequence['ParcelName'] = sequence['ToParcelFinal'].astype(str) + '/' + sequence['ToBlockNumber'].astype(str) + '/' + sequence['ToSubBlockNumber'].astype(str)

sequence.drop(columns = ['BlockNumber', 'SubBlockNumber', 'ToParcelFinal', 'ToBlockNumber', 'ToSubBlockNumber'], inplace = True)

<h1 style="text-align:right;font-size:100%">
הוספת שדה של שם התהליך לטבלת חלקות

In [ ]:
parcel_all['ProcessName'] = parcel_all['TALAR_NUMBER'].astype(str) + '/' + parcel_all['TALAR_YEAR'].astype(str)
parcel_all.drop(columns = ['TALAR_NUMBER', 'TALAR_YEAR'], inplace = True)

<h1 style="text-align:right;font-size:100%">
איכלוס ראשוני של שדות המתארים את התהליך היוצר ותהליך מעדכן
<h1 style="text-align:right;font-size:100%">
חלקות שנוצרו מתהליך שאינו ידוע יוגדרו לכך שנוצרו מהסדר ועל ידי תהליך פיקטיבי שאינו ידוע

In [ ]:
CreateProcessesList = [1, 3, 9, 10, 11] #[תצ"ר, פס"ד, רישום ראשון, תצ"ר לא מוסדר, עריכה חופשית]

CreateProcessMap = {1  : 1,  # תצ"ר
                    3  : 2,  # פס"ד
                    9  : 3,  # הסדר
                    11 : 4} # תצ"ר בשטח לא מוסדר

CreateProcessValues = list(CreateProcessMap.values())


sequence['CreatedByRecord'] = np.where(sequence['ProcessType'].isin(CreateProcessesList),
                                       sequence['GlobalID'],
                                       CNFG.FabricatedCreateProcessGUID)

sequence['CreateProcessType'] = np.where(sequence['ProcessType'].isin(CreateProcessesList),
                                         sequence['ProcessType'].map(CreateProcessMap),
                                         3)



UpdateProcessesList = [4, 5, 6, 7, 8, 12, 13, 14, 15]

sequence['UpdatedByRecord'] = np.where(sequence['ProcessType'].isin(UpdateProcessesList),
                                       sequence['GlobalID'],
                                       None)


sequence['UpdateProcessType'] = np.where(sequence['ProcessType'].isin(UpdateProcessesList),
                                         sequence['ProcessType'],
                                         None)


del [CreateProcessesList, CreateProcessMap, CreateProcessValues, UpdateProcessesList]

<h1 style="text-align:right;font-size:100%">
הסרת כפילות כאשר התהליך היוצר האחרון הוא הקבוע

In [ ]:
created_frame = sequence[['ParcelName', 'CPBUniqueID', 'MapYear', 'CreatedByRecord', 'CreateProcessType']].sort_values(['ParcelName', 'CreateProcessType', 'MapYear'])
created_frame = created_frame[~created_frame['CreatedByRecord'].isna()]
created_frame = created_frame.drop_duplicates(['ParcelName'], keep = 'last')
created_frame = created_frame.drop(columns = ['CPBUniqueID', 'MapYear'])

<h1 style="text-align:right;font-size:100%">
הסרת כפילות כאשר התהליך המעדכן האחרון הוא הקבוע

In [ ]:
updated_frame = sequence[['ParcelName', 'CPBUniqueID', 'MapYear', 'UpdatedByRecord', 'UpdateProcessType']].sort_values(['ParcelName', 'UpdateProcessType', 'MapYear'])
updated_frame = updated_frame[~updated_frame['UpdatedByRecord'].isna()]
updated_frame = updated_frame.drop_duplicates(['ParcelName'], keep = 'last')
updated_frame = updated_frame.drop(columns = ['CPBUniqueID', 'MapYear', 'UpdateProcessType'])

In [ ]:
del sequence
parcel_all = parcel_all.drop(columns = 'ProcessName')

<h1 style="text-align:right;font-size:100%">
מיזוג מידע של תהליך יוצר ותהליך מעדכן אל טבלת חלקות

In [ ]:
Parcels2D = parcel_all.merge(created_frame, on = 'ParcelName', how = 'left')
Parcels2D = Parcels2D.merge(updated_frame, on = 'ParcelName', how = 'left')

In [ ]:
Parcels2D['CreatedByRecord'] = Parcels2D['CreatedByRecord'].fillna(CNFG.FabricatedCreateProcessGUID)
Parcels2D['CreateProcessType'] = Parcels2D['CreateProcessType'].fillna(3)

<h1 style="text-align:right;font-size:100%">
שינוי שם שדה למספר חלקה מלא

In [ ]:
Parcels2D.rename(columns = {'ParcelName' : 'Name'}, inplace = True)

<h1 style="text-align:right;font-size:100%">
הוספת 5 מליון למספרי זיהוי של חלקות

In [ ]:
Parcels2D['ParcelUniqueID'] = Parcels2D['ParcelUniqueID'] + 5000000

In [ ]:
ints = ['BlockNumber', 'SubBlockNumber', 'ParcelUniqueID', 'ParcelNumber', 'BlockUniqueID', 'LandType', 'ParcelType', 'StatedAreaUnit', 'CreateProcessType']
Parcels2D[ints] = Parcels2D[ints].astype('Int64')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  16.e Exporting results to product folder')

In [ ]:
csv_dir = CNFG.CSVPath + 'Parcels2DList.csv'
json_dir = CNFG.JsonPath + 'Parcels2D.geojson'


Parcels2D[['ParcelNumber', 'BlockNumber', 'SubBlockNumber']].to_csv(csv_dir, index = False)
gpd.GeoDataFrame(Parcels2D, geometry = 'geometry', crs = 2039).to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([CNFG.BatchPath + "Parcels2D-Save as Feature Class.bat"]);